# **Stochastic Gradient Descent Regression Models for the CASF-2016 K<sub>i</sub> Dataset**

This Jupyter Notebook builds multiple regression models using the Stochastic Gradient Descent ([Géron, 2023](https://www.isbns.net/isbn/9781098125974/)) method for data from the CASF-2016 K<sub>i</sub> dataset ([de Azevedo et al., 2024](https://doi.org/10.1002/jcc.27449)). This code employs the [SGDRegressor](https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.SGDRegressor.html) class from the [Scikit-Learn](https://scikit-learn.org/stable/) library ([Pedregosa et al., 2011](https://doi.org/10.48550/arXiv.1201.0490)) to generate multiple regression models obtained from the combination of features. It evaluates the regression model's predictive performance using metrics recommended by [Walsh et al., 2021](https://doi.org/10.1038/s41592-021-01205-4) and [de Azevedo et al., 2024](https://doi.org/10.1002/jcc.27449).
<br></br>
<img src="https://drive.usercontent.google.com/download?id=1tlr-fYknxHQri6XSQmy7fHW_jrQPrGpl&export=view&authuser=0" width=600 alt="PDB: 2A4L">
<br><i>Structure of a protein-ligand complex ([de Azevedo et al., 1997](https://doi.org/10.1111/j.1432-1033.1997.0518a.x)) with an inhibitor bound to the macromolecule (PDB access code: [2A4L](https://www.rcsb.org/structure/2A4L)).</i></br>
<br></br>
**References**
<br> </br>
De Azevedo WF, Leclerc S, Meijer L, Havlicek L, Strnad M, Kim SH. Inhibition of cyclin-dependent kinases by purine analogues: crystal structure of human cdk2 complexed with roscovitine. Eur J Biochem. 1997; 243(1-2): 518-26.
PMID: 9030780.
[DOI: 10.1111/j.1432-1033.1997.0518a.x](https://doi.org/10.1111/j.1432-1033.1997.0518a.x)
<br></br>
De Azevedo WF Jr, Quiroga R, Villarreal MA, da Silveira NJF,
Bitencourt-Ferreira G, da Silva AD, Veit-Acosta M, Oliveira PR, Tutone M, Biziukova N, Poroikov V, Tarasova O, Baud S. SAnDReS 2.0: Development of machine-learning models to explore the scoring function space. J Comput Chem. 2024; 45(27): 2333-2346.
PMID: 38900052. [DOI: 10.1002/jcc.27449](https://doi.org/10.1002/jcc.27449) [PubMed](https://pubmed.ncbi.nlm.nih.gov/38900052/)
<br></br>
Géron, A. Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow 3e: Concepts, Tools, and Techniques to Build Intelligent Systems, 3rd ed.; O’Reilly Media: Sebastopol, CA, 2023.
[ISBN: 978-1-098-12597-4](https://www.isbns.net/isbn/9781098125974/)
<br></br>
Pedregosa F, Varoquaux G, Gramfort A, Michel V, Thirion B, Grisel O, Blondel M, Prettenhofer P, Weiss R, Dubourg V, Verplas J, Passos A, Cournapeau D, Brucher M, Perrot M, Duchesnay E. Scikitlearn: Machine Learning in Python. J Mach Learn Res., 2011; 12:2825–2830. [DOI: 10.48550/arXiv.1201.0490](https://doi.org/10.48550/arXiv.1201.0490)
<br></br>
Walsh I, Fishman D, Garcia-Gasulla D, Titma T, Pollastri G; ELIXIR Machine Learning Focus Group; Harrow J, Psomopoulos FE, Tosatto SCE. DOME: recommendations for supervised machine learning validation in biology. Nat Methods. 2021; 18(10): 1122-1127. [DOI: 10.1038/s41592-021-01205-4](https://doi.org/10.1038/s41592-021-01205-4) [PubMed](https://pubmed.ncbi.nlm.nih.gov/34316068/)
<br></br>

It follows the complete Python code.




Generating 1140 regression models...done!

Number of generated regression models:  1140

Metrics saved to file: regression_metrics_C_20_17.csv

Minimum RMSE: 1.4924 for features: ['Gauss 1', 'Gauss 2', 'Repulsion', 'Hydrogen', 'Torsional', 'Q', 'Average Q', 'C', 'N', 'O', 'S', 'P', 'F', 'Cl', 'Br', 'Receptor B-factor(A2)', 'B-factor ratio (Ligand/Receptor)']

Saving scatter plot to file: scatter_plot_test_set_C_20_17.pdf...

done!


In [2]:
#!/usr/bin/env python3
#
################################################################################
# Dr. Walter F. de Azevedo, Jr.                                                #
# [Scopus](https://www.scopus.com/authid/detail.uri?authorId=7006435557)       #
# [GitHub](https://github.com/azevedolab)                                      #
# July 20, 2024                                                                #
################################################################################
#
################################################################################
# Import section                                                               #
################################################################################
import pandas as pd
import math
import numpy as np
from itertools import combinations
from sklearn import preprocessing
from sklearn.linear_model import SGDRegressor

################################################################################
# Define cross_validation() function                                           #
################################################################################
# Function to set up k-fold class. Kfold class to build a n fold
# cross-validation loop and test the generalization ability of regression. With
# cross-validation, we generally obtain a more conservative estimate(that is,
# the error is larger). The cross-validation estimate is a better estimate of
# how well we could generalize to predict on unseen data.
#
################################################################################
# Reference                                                                    #
################################################################################
# Coelho LP, Richert W. (2015) Building Machine Learning Systems with
# Python. 2nd ed. Packt Publishing Ltd. Birmingham UK. 301 pp. See page 162
# (Cross-validation for regression)
def cross_validation(model,X,y,n_splits,random_state,verbose):

    # Import section
    from sklearn.model_selection import KFold, cross_val_score
    from warnings import simplefilter
    import numpy as np

    # Set up k-fold class
    kf = KFold(n_splits=n_splits,shuffle=True, random_state=random_state)

    # Ignore all future warnings
    simplefilter(action='ignore', category=DeprecationWarning)

    # Looping through kf.split()
    for train,test in kf.split(X):

        # Generate regression model
        model.fit(X[train],y[train])

    # Show Walsh metrics if requestes
    if verbose:
        # Show average coefficient of determination using n-fold crossvalidation
        scores = cross_val_score(model,X,y,cv=kf)
        msg_out = "Average coefficient of determination using n-fold "
        msg_out += "cross-validation"
        print("\n"+msg_out+":",np.mean(scores))

    # Return model
    return model

################################################################################
# Define Walsh_and_de_Azevedo_regression_metrics() function                    #
################################################################################
def Walsh_and_de_Azevedo_regression_metrics(y,y_pred):
    """Function to calculate regression metrics as recommended by
    [Walsh et 2021](https://pubmed.ncbi.nlm.nih.gov/34316068/)
    and
    [de Azevedo Jr et al., 2024](https://pubmed.ncbi.nlm.nih.gov/38900052/))

    Parameters:
    y: array with experimental data (dimensions: (n,) )
    y_pred: array with predicted values (dimensions: (n,) )

    where n is the number of instances
    """

    # Import section for this function
    from sklearn.metrics import median_absolute_error
    from scipy import stats
    import numpy as np

    # Determine metrics
    n = len(y)
    y_bar = y.mean()
    rss = np.sum( np.square(y_pred - y) )
    rmse =  np.sqrt( (np.sum( (y - y_pred)**2 ))/n )
    ss_res = np.sum( (y - y_pred)**2 )
    ss_tot = np.sum( (y - y_bar)**2 )
    R2 = 1 - ss_res/ss_tot
    mae = median_absolute_error(y,y_pred)
    r_pearson,p_pearson = stats.pearsonr(y,y_pred)
    rho,p_spearman = stats.spearmanr(y,y_pred)
    core_dome = rmse**2 + mae**2 + (R2 -1)**2
    dome =    np.sqrt( core_dome )
    edomer2 = np.sqrt( core_dome + (r_pearson**2 - 1)**2 )
    edomerho =np.sqrt( core_dome + (rho**2 - 1)**2 )
    edome =   np.sqrt( core_dome + (r_pearson**2 - 1)**2 + (rho**2 - 1)**2 )

    # Return metrics
    return r_pearson,p_pearson,rho,p_spearman,rss,rmse,mae,R2,dome,edomer2,\
    edomerho,edome

################################################################################
# Define scatter_plot() function                                               #
################################################################################
def scatter_plot(y,y_pred,data_in,color_in,color_reg,plt_out,x_l,y_l,title,
                 xlim,ylim,dpi):
    """Function to generate a basic scatter plot"""

    # Import section
    import numpy as np
    import matplotlib.pyplot as plt

    # Create basic scatterplot
    plt.plot(y, y_pred,data_in,color=color_in)

    # Obtain m (slope) and b(intercept) of SGDRegressor line
    m, b = np.polyfit(y,y_pred, 1)

    y_in = np.linspace(xlim[0], xlim[1], 100)

    # Use color_reg as color for regression line
    plt.plot(y_in, m*y_in+b, color=color_reg)
    plt.xlabel(x_l,fontsize = 14)
    plt.ylabel(y_l,fontsize = 14)
    plt.title(title,fontsize = 12)
    plt.xlim( xlim )
    plt.ylim( ylim )
    plt.grid()
    plt.savefig(plt_out,dpi=dpi)
    plt.show()

# Define main() function
def main():

    ############################################################################
    # Training set                                                             #
    ############################################################################
    file_in = "CASF-2016_Ki_training.csv"        # Update here for a new dataset
    print("\nDownloading and preprocessing file: "+file_in,end="...")
    url = "https://bit.ly/CASF-2016_Ki_training" # Update here for a new dataset
    training_set_data = pd.read_csv(url,delimiter=",")
    y_train = training_set_data.iloc[:,8]        # Update here for a new dataset
    print("done!")

    ############################################################################
    # Test set                                                                 #
    ############################################################################
    file_in = "CASF-2016_Ki_test.csv"           # Update here for a new dataset
    print("\nDownloading and preprocessing file: "+file_in,end="...")
    url = "https://bit.ly/CASF-2016_Ki_test"    # Update here for a new dataset
    test_set_data = pd.read_csv(url,delimiter=",")
    y_test = test_set_data.iloc[:,8]            # Update here for a new dataset
    print("done!")

    ############################################################################
    # Define additional inputs
    ############################################################################
    plt_scatter_X_label_str,plt_scatter_y_label_str = "pK$_i$ (Experimental)",\
                        "pK$_i$ (Predicted)"            # Define labels for axes
    plt_dpi = 1500                                      # Plot dpi

    ############################################################################
    # Build a regression model with SGDRegressor
    ############################################################################
    # Set up variables
    n_models = 0
    l = 17     # Cn,l combinations, where n is the number of features in list_in
    lst_in = ["Gauss 1","Gauss 2","Repulsion","Hydrophobic","Hydrogen",
    "Torsional","Torsions","Q","Average Q","C","N","O","S","P","F","Cl","Br",
    "Ligand B-factor(A2)","Receptor B-factor(A2)",
    "B-factor ratio (Ligand/Receptor)"]
    y_list = [y_train,y_test]
    data_list = ["Training Set","Test Set"]
    data_out="Features,Data,n,r,p-value(r),r2,rho,p-value(rho),RSS,RMSE,MAE,R2,"
    data_out += "DOME,EDOMEr2,EDOMErho,EDOME\n"
    data_features = []

    # Calculate the number of combinations (Cn,l)
    n = len(lst_in)
    C_n_l = int(math.factorial(n)/( math.factorial(l)*math.factorial(n - l) ))
    rmse_array = np.zeros(C_n_l)

    # Get all combinations of elements in lst_in and length l
    comb_lst_in = combinations(lst_in, l)

    # Show message
    msg_out = "\nGenerating "+str(C_n_l)+" regression models"
    print(msg_out,end="...")

    # Looping through comb_lst_in
    for feature_list in list(comb_lst_in):

        # Make it a list
        feature_list = list(feature_list)
        data_features.append(feature_list)

        try:
            # For training set
            X_train = training_set_data[feature_list]
            scaler_train = preprocessing.StandardScaler().fit(X_train)
            X_train = scaler_train.transform(X_train)

            # For test set
            X_test = test_set_data[feature_list]
            scaler_test = preprocessing.StandardScaler().fit(X_test)
            X_test = scaler_test.transform(X_test)
            n_models += 1

            # Instantiate an object of SGDRegressor() class
            lin_reg = SGDRegressor(
                        loss="squared_error",
                        penalty="l2",
                        alpha=0.0001,
                        fit_intercept=True,
                        max_iter=100000,
                        tol=1e-5,
                        shuffle=True,
                        verbose=0,
                        validation_fraction=0.2,
                        random_state=46
                ).fit(X_train, y_train)

            # Call cross_validation() function
            lin_reg = cross_validation(lin_reg,X_train,y_train,5,46,False)
            y_train_pred_reg = lin_reg.predict(X_train)
            y_test_pred_reg = lin_reg.predict(X_test)
            y_pred_list = [y_train_pred_reg,y_test_pred_reg]

            # Looping through data
            for count,y in enumerate(y_list):

                # Call Walsh_and_de_Azevedo_regression_metrics() function
                r_pearson,p_pearson,rho,p_spearman,rss,rmse,mae,R2,dome,\
                edomer2,edomerho,edome = \
                Walsh_and_de_Azevedo_regression_metrics(y_list[count],
                                                            y_pred_list[count])

                # Set up output line
                line_out = str(feature_list).replace("'","").replace("[","").\
                                                replace("]","").replace(","," ")
                line_out += ","+data_list[count]+","+str(len(y))+","
                line_out += "{:.4f}".format(r_pearson)+","
                line_out += "{:.4e}".format(p_pearson)+","
                line_out += "{:.4f}".format(r_pearson**2)+","
                line_out += "{:.4f}".format(rho)+","
                line_out += "{:.4e}".format(p_spearman)+","
                line_out += "{:.4e}".format(rss)+","
                line_out += "{:.4f}".format(rmse)+","+"{:.4f}".format(mae)+","
                line_out += "{:.4f}".format(R2)+","+"{:.4f}".format(dome)+","
                line_out += "{:.4f}".format(edomer2)+","
                line_out += "{:.4f}".format(edomerho)+","
                line_out += "{:.4f}".format(edome)

                # Get test set
                if count > 0:

                    # Add test set rmse to the array rmse_array
                    rmse_array[n_models-1] = rmse

            # Update data_out
            data_out += line_out+"\n"

        except:
            print("\nError I didn't generate a regression model for ",
                                                                feature_list)

    print("done!")

    # Open a new file and write data_out and close it
    c_out = str(len(lst_in))+"_"+str(l)
    reg_out = "regression_metrics_C_"+c_out+".csv"
    fo_out = open(reg_out,"w")
    fo_out.write(data_out)
    fo_out.close()
    print("\nNumber of generated regression models: ",n_models)
    print("\nMetrics saved to file: "+reg_out)

    # Find minimum rmse
    min_i = np.argmin(rmse_array)
    best_features = data_features[min_i]
    best_rmse = rmse_array[min_i]
    msg_out = "\nMinimum RMSE: {:.4f}".format(best_rmse)
    msg_out += " for features: "+str(best_features)
    print(msg_out)

    # Regression model for the best set of features
    # For training set
    X_train = training_set_data[best_features]
    scaler_train = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler_train.transform(X_train)

    # For test set
    X_test = test_set_data[best_features]
    scaler_test = preprocessing.StandardScaler().fit(X_test)
    X_test = scaler_test.transform(X_test)

    # Instantiate an object of SGDRegressor() class
    lin_reg = SGDRegressor(
        loss="squared_error",
        penalty="l2",
        alpha=0.0001,
        fit_intercept=True,
        max_iter=100000,
        tol=1e-5,
        shuffle=True,
        verbose=0,
        validation_fraction=0.2,
        random_state=46
        ).fit(X_train, y_train)

    # Call cross_validation() function
    lin_reg = cross_validation(lin_reg,X_train,y_train,5,46,False)
    y_train_pred_reg = lin_reg.predict(X_train)
    y_test_pred_reg = lin_reg.predict(X_test)

    # Call scatter_plot() function for the best regression model
    plot_out = "scatter_plot_test_set_C_"+c_out+".pdf"
    print("\nSaving scatter plot to file: "+plot_out,end="...")
    plt_scatter_title_str = "CASF-2016 K$_i$ (Test Set)"
                                                 # Update here for a new dataset
    plt_scatter_title_str += " (RMSE: {:.4f}".format(best_rmse)+")"
    xlim = (2.0, 12.0)                           # Update here for a new dataset
    ylim = (2.0, 12.0)                           # Update here for a new dataset
    scatter_plot(y_test,y_test_pred_reg,".","blue","black",plot_out,
    plt_scatter_X_label_str,plt_scatter_y_label_str,plt_scatter_title_str,xlim,
             ylim,plt_dpi)
    print("done!")

# Call main() function
main()